<a href="https://colab.research.google.com/github/Huertas97/Sentiment_Analysis/blob/main/sst2_models/DL/SST2_DL_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Regarding the Deep Learning models, all the models used are Transformer-based modelsfrom Hugging Face Transformers library. For the English SST2 binary classification task,the Transformer-based model explored are XLM-RoBERTa base size, DistilBert multilingualcased base size, DistilRoBERTa base size, and DistilBert base size fine-tuned for NLI and STSBenchmark tasks.

SST2 has: 

| Train | Dev | Test |
|-------|-----|------|
| 67349 | 872 | 1821 |

0 = Negative

1 = Positive

In [ ]:
!pip install --upgrade wandb
!pip install transformers
!pip install seqeval
!pip install tensorboardx
!pip install -U simpletransformers
!pip install tqdm

     |████████████████████████████████| 1.8MB 7.7MB/s 
     |████████████████████████████████| 133kB 33.0MB/s 
     |████████████████████████████████| 102kB 11.9MB/s 
     |████████████████████████████████| 163kB 24.8MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6490 sha256=583c297535bee54695ada20803efac4b3be369059a7bab9a130afbba65a55ba5
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built subprocess32
     |████████████████████████████████| 1.5MB 9.8MB/s 
     |████████████████████████████████| 890kB 50.0MB/s 
     |████████████████████████████████| 2.9MB 58.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e2ffbadfcf0d2d7b47e20b58b1139f352dfcf142eb6f83b9d4c3c1d4961e6bef
  Stored in directory: /root/.cache/pip/

# Load the SST2 data 

Clone the GitHub repository

In [ ]:
# Clone the repository and all the dependencies
!git clone https://github.com/Huertas97/Sentiment_Analysis.git

Cloning into 'Sentiment_Analysis'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 8), reused 23 (delta 4), pack-reused 0
Unpacking objects: 100% (30/30), done.


In [ ]:
import io
import pandas as pd
# Load the data from SST2
def loadFile(fpath):
        sst_data = {'X': [], 'y': []}
        with io.open(fpath, 'r', encoding='utf-8') as f:
            for line in f:
                    sample = line.strip().split('\t')
                    sst_data['y'].append(int(sample[1]))
                    sst_data['X'].append(sample[0])
        assert max(sst_data['y']) == 2 - 1
        return sst_data

In [ ]:
sst2_train = loadFile("/content/Sentiment_Analysis/sst_2_data/sentiment-train")
sst2_df_train = pd.DataFrame( {"text": sst2_train["X"], 
               "labels": sst2_train["y"]} )

sst2_dev = loadFile("/content/Sentiment_Analysis/sst_2_data/sentiment-dev")
sst2_df_dev = pd.DataFrame( {"text": sst2_dev["X"], 
               "labels": sst2_dev["y"]} )

sst2_test = loadFile("/content/Sentiment_Analysis/sst_2_data/sentiment-test")
sst2_df_test = pd.DataFrame( {"text": sst2_test["X"], 
               "labels": sst2_test["y"]} )

## Is the data balanced?

In [ ]:
sst2_df_train.groupby(["labels"], as_index=False).agg("count")

,labels,text
0,0,29780
1,1,37569


In [ ]:
sst2_df_dev.groupby(["labels"], as_index=False).agg("count")

,labels,text
0,0,428
1,1,444


In [ ]:
sst2_df_test.groupby(["labels"], as_index=False).agg("count")

,labels,text
0,0,912
1,1,909


Here you can use any model from simpletransformers and train it. The hyperparameters and models available to train and how to train them is available at [Simpletransformers docs](https://simpletransformers.ai/)

In [ ]:
import logging

import pandas as pd
import sklearn

import wandb
from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)

sweep_config = {
    "name": "xlm-roberta-base_2",
    "method": "bayes",  # grid, random
    # "metric": {"name": "train_loss", "goal": "minimize"},
    "metric": {"name": "accuracy", "goal": "maximize"},
    "parameters": {
        "num_train_epochs": {  
                             "values" : [2] #10
        },
        "learning_rate": {
            "min":0, "max":1e-5 #  5e-5
        }, 
        'weight_decay': {
            "min":0.0005, "max":0.01 # 0.05, 0.01
        },

        "train_batch_size" : {"values":[32]},
        "gradient_accumulation_steps": {"values": [3, 5]},
        "scheduler" : {"values": ["constant_schedule_with_warmup",
                                  "polynomial_decay_schedule_with_warmup",
                                  "cosine_schedule_with_warmup"
                                  ]}
    },
    "early_terminate": {"type": "hyperband", "min_iter": 6,}
}

sweep_id = wandb.sweep(sweep_config, project="SST2-pruebas")


# Logger
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Args
model_args = ClassificationArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.max_seq_length = 256
model_args.manual_seed = 4
model_args.use_multiprocessing = True
model_args.train_batch_size = 16 # Train = 63749 / 16 = 42010 batches in 1 epoch
model_args.eval_batch_size = 8
model_args.evaluate_during_training_verbose = True
model_args.evaluate_during_training = True
model_args.best_model_dir = "outputs/best_model" # lower eval_loss
model_args.evaluate_during_training_steps = 2000
model_args.save_model_every_epoch = False 
model_args.save_eval_checkpoints = False # We dont have enough space
model_args.use_cached_eval_features = True
model_args.train_custom_parameters_only = False # Train specific parameters? No, train all parameters
model_args.gradient_accumulation_steps = 2 # The number of training steps (batches) to execute before performing a optimizer.step(). More = less time 



# model_args.labels_list = ["true", "false"]
model_args.wandb_project = "SST2-train"

def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = ClassificationModel(
        'roberta', 
        "distilroberta-base",
        use_cuda=True,
        args=model_args,
        num_labels=2, 
        sweep_config=wandb.config,
    )

    # Train the model and evalaute. Select best model according to evalaution
    model.train_model(sst2_df_train, 
                      eval_df=sst2_df_dev,  
                      acc=sklearn.metrics.accuracy_score)

    # Evaluate the model
    # model.eval_model(sst2_df_test, acc=sklearn.metrics.accuracy_score, 
                    #  verbose=True)

    # Sync wandb
    wandb.join()


wandb.agent(sweep_id, train)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
INFO:wandb.agents.pyagent:Starting sweep agent: entity=None, project=None, count=None


Create sweep with ID: irk0q576
Sweep URL: https://wandb.ai/huertas_97/SST2-pruebas/sweeps/irk0q576


wandb: Agent Starting Run: xm9f8pc9 with config:
wandb: 	gradient_accumulation_steps: 5
wandb: 	learning_rate: 6.821600492325831e-08
wandb: 	num_train_epochs: 2
wandb: 	scheduler: constant_schedule_with_warmup
wandb: 	train_batch_size: 32
wandb: 	weight_decay: 0.0046927561283649135
wandb: Currently logged in as: huertas_97 (use `wandb login --relogin` to force relogin)


INFO:filelock:Lock 139784044892232 acquired on /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46.lock


...

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'cl

  0%|          | 0/67349 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/2105 [00:00<?, ?it/s]

wandb: Ctrl + C detected. Stopping sweep.


# Train final model

Once we have selected the model and best hyperparameters, we train it and log the metrics in W&B. 

In [ ]:
import logging

import pandas as pd
import sklearn

import wandb
from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)



# Logger
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Args
model_args = ClassificationArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.max_seq_length = 256
model_args.manual_seed = 4
model_args.use_multiprocessing = True
model_args.train_batch_size = 16 # Train = 63749 / 16 = 42010 batches in 1 epoch
model_args.eval_batch_size = 8
model_args.evaluate_during_training_verbose = True
model_args.evaluate_during_training = True
model_args.best_model_dir = "outputs/best_model" # lower eval_loss
model_args.evaluate_during_training_steps = 1000
model_args.save_model_every_epoch = False 
model_args.save_eval_checkpoints = True # We dont have enough space
model_args.use_cached_eval_features = True
model_args.train_custom_parameters_only = False # Train specific parameters? No, train all parameters
model_args.gradient_accumulation_steps = 2 # The number of training steps (batches) to execute before performing a optimizer.step(). More = less time 


model_args.wandb_project = "RTE - Hyperparameter Optimization"
model_args.wandb_kwargs = {"name": "vanilla"}


# Hyperparameters
model_args.learning_rate = 0.0000057
model_args.num_train_epochs = 2
model_args.weight_decay = 0.0005


# Create a TransformerModel
model = ClassificationModel(
        'roberta', 
        "distilroberta-base",
        use_cuda=True,
        args=model_args,
        num_labels=2, 
    )

# Train the model and evalaute. Select best model according to evalaution
model.train_model(sst2_df_train, 
                      eval_df=sst2_df_dev,  
                      acc=sklearn.metrics.accuracy_score)

# Evaluate the model
model.eval_model(sst2_df_test, acc=sklearn.metrics.accuracy_score, 
                 verbose=True)



INFO:filelock:Lock 140584132087480 acquired on /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46.lock


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

INFO:filelock:Lock 140584132087480 released on /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46.lock
INFO:filelock:Lock 140584132221528 acquired on /root/.cache/huggingface/transformers/7a0115a4c463f49bc7ab011872fc4a4b81be681a0434075955d29ac3388e225b.a6127d76576e81475313180aceb31a8688f7a649b80e380d26b5d30302dc83c1.lock


Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

INFO:filelock:Lock 140584132221528 released on /root/.cache/huggingface/transformers/7a0115a4c463f49bc7ab011872fc4a4b81be681a0434075955d29ac3388e225b.a6127d76576e81475313180aceb31a8688f7a649b80e380d26b5d30302dc83c1.lock
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weight

...

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


Training loss,0.03752
lr,0.0
global_step,4210
_step,89
_runtime,1337
_timestamp,1609407428
tp,410
tn,379
fp,49
fn,34
mcc,0.80993


Training loss,█▆▃▇▃▄▂▃▂▃▃▂▄▃▂▂▂▁▁▁▁▁▁▁▂▁▂▃▂▂▂▃▁▂▂▃▂▁▃▁
lr,▂▅████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
tp,▅▁▂█▅▅
tn,▁██▅▅▆
fp,█▁▁▄▄▃
fn,▅█▇▁▄▄
mcc,▁▇█▇▆▇


INFO:simpletransformers.classification.classification_model:{'mcc': 0.8635984931439648, 'tp': 867, 'tn': 829, 'fp': 83, 'fn': 42, 'acc': 0.9313563975837452, 'eval_loss': 0.18159797959940574}


({'acc': 0.9313563975837452,
  'eval_loss': 0.18159797959940574,
  'fn': 42,
  'fp': 83,
  'mcc': 0.8635984931439648,
  'tn': 829,
  'tp': 867},
 array([[ 0.57324219, -0.64160156],
        [ 2.59570312, -2.45507812],
        [-0.60107422,  0.55175781],
        ...,
        [-3.25976562,  3.1953125 ],
        [-2.23242188,  2.33398438],
        [-1.49023438,  1.68359375]]),
  <simpletransformers.classification.classification_utils.InputExample at 0x7fdbd4f9c668>])

In [ ]:
def f1_multiclass(labels, preds):
    return sklearn.metrics.f1_score(labels, preds)
result, model_outputs, wrong_predictions = model.eval_model(sst2_df_test, 
                                                            acc=sklearn.metrics.accuracy_score, 
                                                            f1 = f1_multiclass,
                                                            verbose=True)

INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_roberta_256_2_1821


Running Evaluation:   0%|          | 0/228 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


_step,2
_runtime,1
_timestamp,1609411805


_step,▁▅█
_runtime,▁▁▁
_timestamp,▁▁▁


INFO:simpletransformers.classification.classification_model:{'mcc': 0.8635984931439648, 'tp': 867, 'tn': 829, 'fp': 83, 'fn': 42, 'acc': 0.9313563975837452, 'eval_loss': 0.18159797959940574, 'f1': 0.9327595481441635}


In [ ]:
predictions, raw_outputs = model.predict(["HI my friend"])
print(predictions, raw_outputs)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1] [[-2.21484375  1.97949219]]


# Visualizations and ELI5 examples

In [ ]:
!pip install transformers
!pip install -U simpletransformers
!pip install eli5

     |████████████████████████████████| 1.8MB 20.6MB/s 
     |████████████████████████████████| 890kB 59.0MB/s 
     |████████████████████████████████| 2.9MB 58.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7cf58b6c085a7c6663c6325f55d8d3d4af0d97456be02a9b8d8bf609eb56e022
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 215kB 13.8MB/s 
     |████████████████████████████████| 7.5MB 33.9MB/s 
     |████████████████████████████████| 1.2MB 56.0MB/s 
     |████████████████████████████████| 2.0MB 57.7MB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
     |████████████████████████████████| 317kB 54.5MB/s 
     |████████████████████████████████| 51kB 9.9MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 163kB 59.6MB/s 
     |████████████████████████████

In [ ]:
path = "/content/drive/MyDrive/Datos_fake_news/Sentiment_Analysis/SST2/models"

from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)

model_roberta = ClassificationModel("roberta", path, use_cuda=True, args={"no_cache":True})

In [ ]:
model_roberta_no_cuda =  ClassificationModel("roberta", path, use_cuda=False, args={"no_cache":True})

predict_proba : callable
    Black-box classification pipeline. predict_proba
    should be a function which takes a list of strings (documents)
    and return a matrix of shape (n_samples, n_classes) with
    probability values - a row per document and a column per output
    label.

## ELI5

https://towardsdatascience.com/adding-interpretability-to-multiclass-text-classification-models-c44864e8a13b

In [ ]:
import torch.nn as nn
import torch

def predict_proba(sentence):
  result,logits = model_roberta.predict(sentence)
  logits = torch.tensor(logits)
  m = nn.Softmax(dim=1)
  prob = m(logits)
  return prob
results = predict_proba(["Hi friend"])
results.shape

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 2])

In [ ]:
te = TextExplainer(random_state=42, n_samples = 1000)

In [ ]:
import eli5
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42, n_samples =500)
te.fit("I will find you and I will kill you", predict_proba=predict_proba,  )
h = te.show_prediction(target_names=["Negative", "Positive"])
h

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
from IPython.display import display, HTML
 
type(display(HTML(h.data)))

NoneType

In [ ]:
print(te.metrics_)

{'mean_KL_divergence': 0.009694694194737053, 'score': 0.9967585753109094}


In [ ]:
te.show_prediction()